In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os

In [39]:
train = pd.read_csv("./wind/train.csv")
test = pd.read_csv("./wind/test.csv")
submission = pd.read_csv("./wind/sample_submission.csv")

In [119]:
test_pred = pd.read_csv('./wind/test.csv').drop(columns=['id', 'pressure', 'snowing', 'precipitation'])

In [40]:
train.drop(['id'], axis=1, inplace=True)

In [41]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [42]:
x = train.drop(['pressure', 'snowing', 'precipitation', 'target'], axis = 1)
y = train['target']

In [72]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [73]:
# 더미 변환
X_train=pd.get_dummies(X_train)
X_test=pd.get_dummies(X_test)

In [88]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15420 entries, 5489 to 15795
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   temperature     15420 non-null  category
 1   humidity        15420 non-null  category
 2   wind_speed      15420 non-null  category
 3   wind_direction  15420 non-null  category
 4   cloudiness      15420 non-null  category
dtypes: category(5)
memory usage: 680.8 KB


In [74]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15420, 5), (3855, 5), (15420,), (3855,))

In [76]:
from sklearn.preprocessing import LabelEncoder
col=['temperature', 'humidity', 'wind_speed', 'wind_direction', 'cloudiness']
encoder=LabelEncoder()
for i in col:
    X_train[i]=encoder.fit_transform(X_train[i])
    X_test[i]=encoder.fit_transform(X_test[i])
    X_train[i]=X_train[i].astype('category')
    X_test[i]=X_test[i].astype('category')

In [60]:
normal_cols = X_train.columns
normal_cols

Index(['temperature', 'humidity', 'wind_speed', 'wind_direction',
       'cloudiness'],
      dtype='object')

In [65]:
enc_tr = pd.DataFrame(
    enc.fit_transform(X_train[normal_cols]).toarray(), #변환
    columns = enc.get_feature_names_out() 
)
enc_te = pd.DataFrame(
    enc.fit_transform(X_test[normal_cols]).toarray(), #변환
    columns = enc.get_feature_names_out() 
)

In [81]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
import lightgbm as lgb



sc = StandardScaler()
rc = RobustScaler()

In [82]:
x_train = sc.fit_transform(X_train)
x_test = sc.fit_transform(X_test)

In [48]:
x1_train = rc.fit_transform(X_train)
x1_test = rc.fit_transform(X_test)

In [83]:
modelV0 = DecisionTreeRegressor(max_depth=5, random_state=42)
modelV0.fit(x_train, y_train)
modelV0.score(x_train, y_train), modelV0.score(x_test, y_test)

(0.17567317379776104, 0.13719090852003757)

In [84]:
modelV1 = RandomForestRegressor(n_estimators = 100, random_state=42)
modelV1.fit(x_train, y_train)
modelV1.score(x_train, y_train), modelV1.score(x_test, y_test)

(0.8944768604137762, 0.2062979114714374)

In [123]:
from sklearn.ensemble import GradientBoostingRegressor
hp = {
    "random_state" : 42,
    "max_depth" : 1,
    "n_estimators" : 100
}

gradient = GradientBoostingRegressor(**hp).fit(X_train, y_train)
print(f'훈련용 평가지표: {gradient.score(X_train, y_train)} / 테스트용 평가지표: {gradient.score(X_test, y_test)}')

훈련용 평가지표: 0.16276037100522733 / 테스트용 평가지표: 0.08217575691883816


In [126]:
y_pred = modelV1.predict(test_pred)

C:\Users\Playdata\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [127]:
y_pred

array([7.74302, 8.03034, 7.92399, ..., 7.8714 , 8.24119, 8.41386])

In [85]:
from xgboost import XGBClassifier,plot_importance

hp = {
    "random_state" : 42
}

xgb = XGBClassifier(**hp).fit(x_train, y_train)
print(f'훈련용 평가지표: {xgb.score(x_train, y_train)} / 테스트용 평가지표: {xgb.score(x_train, y_test)}')

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [   0    1    2 ... 8127 8128 8129], got [ 0.     0.243  0.26  ... 17.125 17.256 17.344]

In [87]:
y_train

5489      3.595
12559     8.142
18718     8.182
5230      3.927
8568      9.080
          ...  
11284     7.492
11964     6.646
5390      4.451
860       6.057
15795    12.172
Name: target, Length: 15420, dtype: float64

In [52]:
from lightgbm import LGBMClassifier, plot_importance

In [80]:
hp = {
    "random_state" : 42,
    "max_depth" : 2,
    "n_estimators" : 100,
    "learning_rate": 0.01,
}

lgb = LGBMClassifier(**hp).fit(X_train, y_train)
print(f'훈련용 평가지표: {lgb.score(X_train, y_train)} / 테스트용 평가지표: {lgb.score(X_test, y_test)}')

ValueError: Unknown label type: 'continuous'

In [55]:
x_train

array([[-1.25951573,  0.78488313,  0.23998162, -0.24028098, -1.00512635],
       [ 0.90746367,  0.89259992, -0.64249451, -0.28195895, -0.1548011 ],
       [ 0.64745959, -1.58488642,  0.68121968, -1.81015094, -0.55161955],
       ...,
       [-1.28376048,  1.25614412,  0.53414033, -0.66400695,  0.07195229],
       [ 0.63882065, -0.25189105,  0.53414033, -0.72305073,  1.14903094],
       [ 0.26400233,  1.16189192, -0.93665321,  0.43698593,  0.07195229]])

In [35]:
modelV3 = DecisionTreeRegressor(max_depth=5, random_state=42)
modelV3.fit(x1_train, y_train)
modelV3.score(x1_train, y_train), modelV3.score(x_test, y_test)

(0.17567317379776093, 0.07055564675679438)

In [36]:
modelV4 = RandomForestRegressor(n_estimators = 100, random_state=42)
modelV4.fit(x_train, y_train)
modelV4.score(x_train, y_train), modelV4.score(x_test, y_test)

(0.8941514844876459, 0.2239250441404732)

In [ ]:
modelV2 = lgb.LGBMClassifier(num_leaves=31, objective='binary')
modelV2.fit(X_train, y_train)
y_pred = modelV2.predict(X_test)
accuracy_score(y_test, y_pred)

In [95]:
ids = test['id']
predictions = gradient.predict(test.drop['id'], axis=1)

submission = pd.DataFrame({'id' : ids, 'target': predictions})
submission.to_csv('./wind/sample_submission.csv', index=False)

TypeError: 'method' object is not subscriptable

In [105]:
submission = pd.read_csv("./wind/sample_submission.csv")
submission

,id,target
0,TEST_00000,7.33243
1,TEST_00001,7.92765
2,TEST_00002,7.96670
3,TEST_00003,7.27823
4,TEST_00004,7.27823
...,...,...
19270,TEST_19270,7.35143
19271,TEST_19271,7.27823
19272,TEST_19272,7.18007
19273,TEST_19273,7.88582


In [121]:
submission.loc[:, "target"] = y_pred
submission

,id,target
0,TEST_00000,5.034239
1,TEST_00001,5.034239
2,TEST_00002,5.034239
3,TEST_00003,5.034239
4,TEST_00004,5.034239
...,...,...
19270,TEST_19270,5.034239
19271,TEST_19271,5.034239
19272,TEST_19272,5.034239
19273,TEST_19273,5.034239
